In [1]:
library(SnapATAC)

Loading required package: Matrix
Loading required package: rhdf5


### Step 3. Barcode selection

In [2]:
x.sp = createSnap(
	file="all_merge.snap",
	sample="scatac",
	num.cores=15
	);

Epoch: reading the barcode session ...


In [3]:
x.sp

number of barcodes: 1345
number of bins: 0
number of genes: 0
number of peaks: 0

### Step 4. Add cell-by-bin matrix to existing snap object

Here we use cell-by-bin matrix of 5kb resolution as input for clustering

In [4]:
# show what bin sizes exist in snap file
showBinSizes("all_merge.snap");
x.sp = addBmatToSnap(x.sp, bin.size=5000, num.cores=15)

[1]  5000 10000

Epoch: reading cell-bin count matrix session ...


In [5]:
x.sp

number of barcodes: 1345
number of bins: 11383842
number of genes: 0
number of peaks: 0

### Step 5. Matrix binarization

We next convert the cell-by-bin count matrix to a binary matrix. We found some items in the matrix have abnormally high coverage perhaps due to the alignment error. Therefore, we first remove top 0.1% items in the count matrix followed by converting the rest of the values into binary.

In [6]:
x.sp = makeBinary(x.sp, mat="bmat")

### Step 7. Dimensionality Reduction (SnapATAC)
We next run directionality reduction using `runJDA` function which contains the following steps: 1) we convert the filtered genome-wide cell-by-bin matrix into a cell-by-cell similarity matrix by estimating the jaccard index between two cells in the basis of profile overlaps; 2) due to the high dropout rate, we found that the jaccard index is highly affected by the read depth differing between cells. To eliminate such confounding factor, we developed a regression-based method `normOVE` to eliminate such confounding factor; 3) Like other single-cell analysis, snATAC-seq contains extensive technical noise due to the high drop-out rate. To overcome this challenge, we applied PCA or SVD to combine information across a correlated feature set hereby creating a mega-feature and exclude the variance potential resulting from technical noise.

In [7]:
x.sp = runJDA(
	obj=x.sp,
	input.mat="bmat",
	bin.cov.zscore.lower=-2,
	bin.cov.zscore.upper=2,
	pc.num=50,
	norm.method="normOVE",
	max.var=5000,
	do.par=TRUE,
	ncell.chunk=1000,
	num.cores=15,
	seed.use=42,
	tmp.folder=tempdir()
	);

Epoch: checking the inputs ...
Epoch: filtering bins ..
Epoch: running jaccard index matrix ...
Epoch: normalizing jaccard index matrix ...


ERROR: Error in isOpen(con): 链结不对


In [15]:
parallel::detectCores()

[1] 64

### Step 8. Determine statistically significant principal components (SnapATAC)

We next Determine how many PCs to include for downstream analysis. We use an ad hoc method for determining which PCs to use by looking at a plot of the standard deviations of the principle components and draw your cutoff where there is a clear elbow in the graph. The other ad hoc way to determine PCs is to plot out every two PCs and select the number of PCs until there is no obvious structure.

In [ ]:
plotDimReductElbow(
    obj=x.sp, 
    point.size=1.5,
    point.shape=19,
    point.color="red",
    point.alpha=1,
    pdf.file.name=NULL,
    pdf.height=7,
    pdf.width=7,
    labs.title="PCA Elbow plot",
    labs.subtitle=NULL
    );

In [13]:
plotDimReductPW(
    obj=x.sp, 
    pca.dims=1:50,
    point.size=0.3,
    point.color="grey",
    point.shape=19,
    point.alpha=0.6,
    down.sample=5000,
    pdf.file.name=NULL, 
    pdf.height=7, 
    pdf.width=7
    );

ERROR: Error in plotDimReductPW.default(obj = x.sp, pca.dims = 1:50, point.size = 0.3, : dim.reduct is not complete, run 'runDimReduct' first


In [14]:
x.sp = runKNN(
    obj=x.sp,
    pca.dims=2:30,
    weight.by.sd=TRUE,
    k=15
    );

ERROR: Error in runKNN.default(obj = x.sp, pca.dims = 2:30, weight.by.sd = TRUE, : dimentionality reduction is not complete, run 'runDimReduct' first


In [9]:
x.sp

number of barcodes: 1345
number of bins: 11383842
number of genes: 0
number of peaks: 0

In [13]:
x.sp@

Number of cells: 0 
 Number of dims:  0 
 Normalization:  FALSE 
 Normalization method:   